In [1]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import urllib.request
import bs4
import csv
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import pickle
from heapq import heappush, nlargest
import heapq


import parser_utils
import collector_utils
import index_utils
from index_utils import create_inverted_dict

In [2]:
# We take movies1-2-3.html parse them and create a list of links
links = collector_utils.get_links('movies1.html', 'movies2.html', 'movies3.html')

/home/spelvio/Desktop/Homeworks/H3/HW3/collector_utils.py:9: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file /home/spelvio/Desktop/Homeworks/H3/HW3/collector_utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup1 = BeautifulSoup(movies1)
/home/spelvio/Desktop/Homeworks/H3/HW3/collector_utils.py:13: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on li

In [7]:
# create films data frame
df_films = parser_utils.films_data_frame(20, links)

In [8]:
df_films

,film_id,Wikipedia_link,title,intro,plot,film_name,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget
0,0,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Love by the Light of the Moon,Edwin S. Porter,NA,NA,NA,NA,1901 (1901),NA,United States,Silent,NA
1,1,https://en.wikipedia.org/wiki/The_Martyred_Pre...,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",The Martyred Presidents,Edwin S. Porter,NA,NA,See below,NA,1901,1 minutes,United States,English,NA
2,2,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...","Terrible Teddy, the Grizzly King","Terrible Teddy, the Grizzly King is a 1901 Ame...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,Jack and the Beanstalk (1902 film),Jack and the Beanstalk is a 1902 American shor...,In this earliest known adaptation of the class...,Jack and the Beanstalk (1902 film),George S. FlemingEdwin S. Porter,Edison Studios,NA,See below,NA,1902,10 minutes,United States,English,NA
4,4,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent f...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland (1903 film),Cecil HepworthPercy Stow,Cecil M. HepworthHerman Casler (exec. producer...,Lewis Carroll (book)Cecil M. Hepworth,May ClarkCecil M. HepworthMrs. Cecil HepworthN...,NA,17 October 1903 (1903-10-17),approx. 8:19 minutes,United Kingdom,Silent,NA
5,5,https://en.wikipedia.org/wiki/The_Great_Train_...,The Great Train Robbery (1903 film),The Great Train Robbery is a 1903 American sil...,The film opens with two bandits breaking into ...,The Great Train Robbery (1903 film),Edwin S. Porter,Edwin S. Porter,NA,Alfred C. AbadieBroncho Billy AndersonJustus D...,NA,"December 1, 1903 (1903-12-01)",12 minutes (at 18 frame/s),United States,SilentEnglish intertitles,$150[1]
6,6,https://en.wikipedia.org/wiki/The_Suburbanite,The Suburbanite,The Suburbanite is a 1904 American short comed...,The film is about a family who move to the sub...,The Suburbanite,Wallace McCutcheon,NA,Frank Marion,John Troiano,NA,November 1904,NA,United States,Silent,NA
7,7,https://en.wikipedia.org/wiki/The_Little_Train...,The Little Train Robbery,The Little Train Robbery is a parodic sequel t...,The opening scene shows the interior of the ro...,The Little Train Robbery,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,8,https://en.wikipedia.org/wiki/The_Night_Before...,The Night Before Christmas (1905 film),The Night Before Christmas is a 1905 American ...,Scenes are introduced using lines of the poem....,The Night Before Christmas (1905 film),Edwin S. Porter,NA,Clement Clarke Moore (poem),NA,NA,"December 16, 1905 (1905-12-16)",9:00 (800 feet),United States,English,NA
9,9,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,Dream of a Rarebit Fiend (1906 film),Dream of a Rarebit Fiend is a 1906 American si...,The Rarebit Fiend gorges on Welsh rarebit at a...,Dream of a Rarebit Fiend (1906 film),Wallace McCutcheonEdwin S. Porter,NA,Winsor McCay (comic strip),Jack Brawn,NA,1906 (1906),6 minutes,United States,Silent,NA


In [5]:

df_films = pd.read_csv('../data/films_dataFrame.csv')   # read data frame from a file

In [11]:
words_dict = {}
df_films = df_films.apply(create_inverted_dict, axis=1, vocabulary = words_dict)
# da riprovare

In [15]:
len(words_dict)

1095

In [13]:
# create vocabulary
vocabulary = index_utils.create_words_voc(words_dict)

In [14]:
len(vocabulary)

1095

In [16]:
vocabulary

{'love': 0,
 'light': 1,
 'moon': 2,
 '1901': 3,
 'film': 4,
 'edwin': 5,
 'S.': 6,
 'porter': 7,
 'produc': 8,
 'edison': 9,
 'manufactur': 10,
 'compani': 11,
 '1': 12,
 'It': 13,
 'mix': 14,
 'anim': 15,
 'live': 16,
 'action': 17,
 'predat': 18,
 'man': 19,
 'theme': 20,
 '1902': 21,
 'french': 22,
 'scienc': 23,
 'fiction': 24,
 'A': 25,
 'trip': 26,
 'georg': 27,
 'méliès.th': 28,
 'paint': 29,
 'smile': 30,
 'face': 31,
 'hang': 32,
 'park': 33,
 'night': 34,
 'young': 35,
 'coupl': 36,
 'walk': 37,
 'past': 38,
 'fenc': 39,
 'learn': 40,
 'rail': 41,
 'look': 42,
 'the': 43,
 'they': 44,
 'embrac': 45,
 "'s": 46,
 'get': 47,
 'bigger': 48,
 'sit': 49,
 'bench': 50,
 'tree': 51,
 'view': 52,
 'block': 53,
 'caus': 54,
 'frown': 55,
 'In': 56,
 'last': 57,
 'scene': 58,
 'fan': 59,
 'woman': 60,
 'hat': 61,
 'left': 62,
 'sky': 63,
 'perch': 64,
 'shoulder': 65,
 'see': 66,
 'everyth': 67,
 'better': 68,
 'martyr': 69,
 'presid': 70,
 'american': 71,
 'direct': 72,
 'porter.th': 

In [12]:
# save tfldf_vocabulary as a file

# Store data (serialize)
with open('../data/vocabulary.pickle', 'wb') as handle:
    pickle.dump(vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# Store data (serialize)
with open('../data/words_dict.pickle', 'wb') as handle:
    pickle.dump(words_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [18]:
# Load data (deserialize)
with open('../data/words_dict.pickle', 'rb') as handle:
    words_dict = pickle.load(handle)
    
# Load data (deserialize)
with open('../data/vocabulary.pickle', 'rb') as handle:
    vocabulary = pickle.load(handle)


In [7]:
def create_dictionary_tfldf(Vocabulary):
    N_Vocabulary = {}
    with tqdm(total = 228806) as pbar:
        for k in Vocabulary.keys():
            repetition = len(Vocabulary[k])
            IDF = math.log(30000/repetition)
            for elem in Vocabulary[k].keys():
                valore = Vocabulary[k][elem]

                length = list(h[h['film_id'] == elem]['words_number'])[0]

                Tf = (valore)/(length)

                if k not in N_Vocabulary:
                    N_Vocabulary[k] = {elem : Tf*IDF}
                else:
                    N_Vocabulary[k][elem] = Tf*IDF
            pbar.update(1)
    return N_Vocabulary

In [168]:
tfldf_vocabulary = create_dictionary_tfldf(vocabulary)

100%|██████████| 228806/228806 [1:22:22<00:00, 46.30it/s]  


In [169]:
len(tfldf_vocabulary)

228806

In [179]:
# save tfldf_vocabulary as a file

# Store data (serialize)
with open('../data/tfidf_vocabulary.pickle', 'wb') as handle:
    pickle.dump(tfldf_vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# Load data (deserialize)
with open('../data/tfidf_vocabulary.pickle', 'rb') as handle:
    tfidf_voc = pickle.load(handle)

print(tfldf_vocabulary == tfidf_voc)

True


In [8]:
# Load data (deserialize)
with open('../data/tfidf_vocabulary.pickle', 'rb') as handle:
    tfidf_vocabulary = pickle.load(handle)

In [117]:
# search engine 1
def search_engine_1(row):
    cont = 0
    for elem in cleaned_query_1:
        if(elem in vocabulary):
            if(row['film_id'] in vocabulary[elem]):
                cont += 1
    if cont == len(cleaned_query_1):
        return 1
    return 0

In [115]:
se1 = df_films

In [125]:
q = 'Toy Story'
cleaned_query_1 = clean_film(q)
se1['Engine1'] = se1.apply(search_engine_1, axis = 1)
b1 = se1[['title','Wikipedia_link', 'intro', 'Engine1']]

In [126]:
b1[b1['Engine1'] == 1]

,title,Wikipedia_link,intro,Engine1
700,The Last Command (1928 film),https://en.wikipedia.org/wiki/The_Last_Command...,The Last Command is a 1928 silent film directe...,1
1464,Cavalcade (1933 film),https://en.wikipedia.org/wiki/Cavalcade_(1933_...,Cavalcade is a 1933 American epic pre-Code dra...,1
1625,Babes in Toyland (1934 film),https://en.wikipedia.org/wiki/Babes_in_Toyland...,Babes in Toyland is a Laurel and Hardy musical...,1
1704,It Happened One Night,https://en.wikipedia.org/wiki/It_Happened_One_...,It Happened One Night is a 1934 pre-Code Ameri...,1
2433,Rebecca of Sunnybrook Farm (1938 film),https://en.wikipedia.org/wiki/Rebecca_of_Sunny...,Rebecca of Sunnybrook Farm is a 1938 American ...,1
...,...,...,...,...
28078,Happy Wedding,https://en.wikipedia.org/wiki/Happy_Wedding,Happy Wedding is 2016 Indian Malayalam romanti...,1
28079,Aadupuliyattam,https://en.wikipedia.org/wiki/Aadupuliyattam,Aadupuliyattam (English: The Goat and Tiger Ga...,1
28369,Vaisakhi List,https://en.wikipedia.org/wiki/Vaisakhi_List,Vaisakhi List[1]) is a 2016 Punjabi film direc...,1
28729,Pasamalar,https://en.wikipedia.org/wiki/Pasamalar,Pasamalar (transl. The Flower of Love) is a 19...,1


In [109]:
# search engine 2
def cosSim(row):
    film_vector = []
    for elem in cleaned_query:
        if(elem in tfidf_vocabulary):
            if(row['film_id'] in tfidf_vocabulary[elem]):
                film_vector.append(tfidf_vocabulary[elem][row['film_id']])
            else:
                film_vector.append(0)
        else:
            film_vector.append(0)
    
    query_vector_idf = tfidf_query(cleaned_query)
    cos_sim = cosine_similarity([film_vector], [query_vector_idf])[0][0]
    return cos_sim

In [141]:
list_tuples = []
for i in range(30000):
    list_tuples.append((list(h[h['film_id'] == i]['Similarity'])[0], i))

In [143]:
heapq.heapify(list_tuples)
largest_sim = nlargest(15, list_tuples)

In [160]:
res_sim = pd.DataFrame(columns = ['title', 'intro', 'Wikipedia_link', 'Similarity'])
for elem in largest_sim:
    res_sim = res_sim.append(h[h['film_id'] == elem[1]][['title', 'intro', 'Wikipedia_link', 'Similarity']])
res_sim

,title,intro,Wikipedia_link,Similarity
14590,Harry Potter and the Goblet of Fire (film),Harry Potter and the Goblet of Fire is a 2005 ...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.956166
15571,Harry Potter and the Half-Blood Prince (film),Harry Potter and the Half-Blood Prince is a 20...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.955962
15148,Harry Potter and the Order of the Phoenix (film),Harry Potter and the Order of the Phoenix is a...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.948416
13787,Harry Potter and the Philosopher's Stone (film),Harry Potter and the Philosopher's Stone (rele...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.944247
15768,Harry Potter and the Deathly Hallows – Part 1,Harry Potter and the Deathly Hallows – Part 1 ...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.938298
14393,Harry Potter and the Prisoner of Azkaban (film),Harry Potter and the Prisoner of Azkaban is a ...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.914860
20949,Harry Potter and the Deathly Hallows – Part 2,Harry Potter and the Deathly Hallows – Part 2 ...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.900084
15962,Harry Potter and the Deathly Hallows – Part 2,Harry Potter and the Deathly Hallows – Part 2 ...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.900084
13988,Harry Potter and the Chamber of Secrets (film),Harry Potter and the Chamber of Secrets is a 2...,https://en.wikipedia.org/wiki/Harry_Potter_and...,0.879567
20590,The Man Who Cried,The Man Who Cried is a 2000 Anglo-French drama...,https://en.wikipedia.org/wiki/The_Man_Who_Cried,0.816388


In [16]:
h = df_films

In [302]:
query = 'Disney movies 2019'
cleaned_query = clean_film(query)
query_vector = tfidf_query(cleaned_query)
h['Similarity'] = h.apply(cosSim, axis = 1)
b = h[['title','Wikipedia_link', 'intro', 'Similarity']]

[1, 1, 1] [0.008326340115043612, 0.003383447630848224, 0.010524399286434694]


In [90]:
def tfidf_query(q):
    tfidf_q = []
    for elem in q:
        tfidf_q.append(1)
    return tfidf_q

In [303]:
b[b['Similarity'] > 0.8].sort_values('Similarity', ascending = False)

,title,Wikipedia_link,intro,Similarity
12266,Thumbelina (1994 film),https://en.wikipedia.org/wiki/Thumbelina_(1994...,Thumbelina (also known as Hans Christian Ander...,0.927540
25560,Jaanam Samjha Karo,https://en.wikipedia.org/wiki/Jaanam_Samjha_Karo,"Jaanam Samjha Karo (Hindi: जानम समझा करो, Urdu...",0.816363
16094,The Apparition,https://en.wikipedia.org/wiki/The_Apparition,The Apparition is a 2012 American-German super...,0.816363
16430,Frozen (2013 film),https://en.wikipedia.org/wiki/Frozen_(2013_film),Frozen is a 2013 American 3D computer-animated...,0.815474
10614,Benji the Hunted,https://en.wikipedia.org/wiki/Benji_the_Hunted,Benji the Hunted is a 1987 American children's...,0.812547
14276,Scary Movie 3,https://en.wikipedia.org/wiki/Scary_Movie_3,Scary Movie 3 is a 2003 American horror sci-fi...,0.812547
10629,Bride of Boogedy,https://en.wikipedia.org/wiki/Bride_of_Boogedy,"Bride of Boogedy is a 1987 family film, direct...",0.812173
9876,Annie (1982 film),https://en.wikipedia.org/wiki/Annie_(1982_film),Annie is a 1982 American musical comedy-drama ...,0.812173
25221,Jo Jeeta Wohi Sikandar,https://en.wikipedia.org/wiki/Jo_Jeeta_Wohi_Si...,"Jo Jeeta Wohi Sikandar (transl. Whoever wins, ...",0.812173
22259,Mulan (2009 film),https://en.wikipedia.org/wiki/Mulan_(2009_film),Mulan: Rise of a Warrior (simplified Chinese: ...,0.812173


In [167]:
print(list(h[h['film_id'] == 10000]['Starring'])[0].split('\n'))

['Charles Bronson', 'Lisa Eilbacher', 'Andrew Stevens', 'Gene Davis', 'Geoffrey Lewis', 'Wilford Brimley']


# Part 3

In [211]:
import pycountry

# make a list of languages in our dataframe

opt = []
for elem in list(pycountry.languages):
    if elem.name in languages and elem.name not in opt and len(elem.name) > 3:
        opt.append(elem.name)
opt.append('Silent')
opt.sort()

['Afrikaans', 'Albanian', 'Algonquin', 'Arabic', 'Armenian', 'Assamese', 'Awadhi', 'Azerbaijani', 'Beng', 'Benga', 'Bengali', 'Bhojpuri', 'Bosnian', 'Bulgarian', 'Catalan', 'Cheyenne', 'Chinese', 'Cree', 'Croatian', 'Czech', 'Danish', 'Dari', 'Dutch', 'Dzongkha', 'English', 'Fijian', 'Filipino', 'Finnish', 'Fore', 'French', 'Georgian', 'German', 'Gondi', 'Gujarati', 'Haryanvi', 'Hebrew', 'Hiligaynon', 'Hindi', 'Hmong', 'Hung', 'Hungarian', 'Icelandic', 'Indonesian', 'Inuktitut', 'Irish', 'Italian', 'Japanese', 'Kannada', 'Kinyarwanda', 'Korean', 'Krio', 'Lahu', 'Lakota', 'Latin', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Malayalam', 'Mandar', 'Mandari', 'Maori', 'Mara', 'Marathi', "Mi'kmaq", 'Mohawk', 'Mongo', 'Mongol', 'Mongolian', 'Navajo', 'Norwegian', 'Nyanja', 'Pare', 'Pawnee', 'Persian', 'Polish', 'Portuguese', 'Quechua', 'Rajasthani', 'Rema', 'Roma', 'Romanian', 'Rotuman', 'Russian', 'Sanskrit', 'Scots', 'Serbian', 'Serbo-Croatian', 'Shan', 'Sicilian', 'Silent', 'Sinhala', 'Som

In [220]:
# languages in our DataSet
languages = ''
for elem in list(df_films['Language'].drop_duplicates()):
    try:
        l = elem.strip().split()
    except:
        continue
    for lan in l:
        if lan not in languages:
            languages += lan

In [215]:
import ipywidgets as widgets

choose_language = widgets.Dropdown(
    options=opt,
    value='Afrikaans',
    description='Language:',
    disabled=False,
)

In [217]:
choose_language

Dropdown(description='Language:', index=26, options=('Afrikaans', 'Albanian', 'Algonquin', 'Arabic', 'Armenian…

In [241]:
def selected_films(row):
    try:
        if choose_language.value in row['Language']:
            return 1
    except:
        return 0
    return 0

In [233]:
search2 = df_films

In [243]:
search2['sel_lan'] = search2.apply(selected_films, axis = 1)

,Unnamed: 0,film_id,Wikipedia_link,title,intro,plot,film_name,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget,Similarity,Engine1,sel_lan
1,1,1,https://en.wikipedia.org/wiki/The_Martyred_Pre...,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",The Martyred Presidents,Edwin S. Porter,NaN,NaN,See below,NaN,1901,1 minutes,United States,English,NaN,0.00000,0,1
3,3,3,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,Jack and the Beanstalk (1902 film),Jack and the Beanstalk is a 1902 American shor...,In this earliest known adaptation of the class...,Jack and the Beanstalk (1902 film),George S. FlemingEdwin S. Porter,Edison Studios,NaN,See below,NaN,1902,10 minutes,United States,English,NaN,0.00000,0,1
5,5,5,https://en.wikipedia.org/wiki/The_Great_Train_...,The Great Train Robbery (1903 film),The Great Train Robbery is a 1903 American sil...,The film opens with two bandits breaking into ...,The Great Train Robbery (1903 film),Edwin S. Porter,Edwin S. Porter,NaN,Alfred C. AbadieBroncho Billy AndersonJustus D...,NaN,"December 1, 1903 (1903-12-01)",12 minutes (at 18 frame/s),United States,SilentEnglish intertitles,$150[1],0.00000,0,1
8,8,8,https://en.wikipedia.org/wiki/The_Night_Before...,The Night Before Christmas (1905 film),The Night Before Christmas is a 1905 American ...,Scenes are introduced using lines of the poem....,The Night Before Christmas (1905 film),Edwin S. Porter,NaN,Clement Clarke Moore (poem),NaN,NaN,"December 16, 1905 (1905-12-16)",9:00 (800 feet),United States,English,NaN,0.57735,0,1
14,14,14,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,Laughing Gas (film),Laughing Gas is the title of several American ...,The plot is that of a black woman going to the...,Laughing Gas (film),Edwin S. Porter,Abe SternJulius Stern,Tom Buckingham,Bertha RegustusEdward Boulden,NaN,1907 (1907),9 minutes,United States,Silent film English intertitles,NaN,0.00000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29849,29849,29849,https://en.wikipedia.org/wiki/Bharathi_(film),Rhubarb (1969 film),Rhubarb was a 1969 British short film written ...,A Police Inspector and a vicar play a round of...,Rhubarb (1969 film),Eric Sykes,Jon Penington,Eric Sykes,Harry SecombeEric Sykes,Brian Fahey,1969 (1969),37 minutes,United Kingdom,English,NaN,0.57735,0,1
29876,29876,29876,https://en.wikipedia.org/wiki/Rishi_(film),Deep End (film),Deep End is a 1970 British-West German drama f...,"Mike (John Moulder Brown), a 15-year-old dropo...",Deep End (film),Jerzy Skolimowski,Helmut Jedele,Jerzy GruzaJerzy SkolimowskiBoleslaw Sulik,Jane AsherJohn Moulder Brown,Cat StevensCan,1 September 1970 (1970-09-01) (VFF)\n1971 (197...,90 minutes,United KingdomWest Germany,English,NaN,0.00000,0,1
29883,29883,29883,https://en.wikipedia.org/wiki/Little_John_(film),Little John (film),Little John is a 2001 trilingual fantasy film ...,The film opens with a historical story in whic...,Little John (film),Singeetam,Sujatha RangarajanM. VaradarajaK. ManiprasadEx...,Dialogue:-Crazy Mohan (Tamil)Sushma Ahuja (Hin...,Bentley MitchumJyothikaAnupam KherNassar,Pravin Mani,13 April 2001 (2001-04-13),126 minutes,India,TamilHindiEnglish,NaN,0.00000,0,1
29928,29928,29928,https://en.wikipedia.org/wiki/Charlie_Chaplin_...,Family Life (1971 British film),Family Life (US: Wednesday's Child)[2] is a 19...,"A young woman, Janice, is living with her rest...",Family Life (1971 British film),Ken Loach,Tony Garnett,NaN,Sandy RatcliffMalcolm TierneyGrace Cave,Marc Wilkinson,2 December 1971 (1971-12-02) (UK)\n5 October 1...,108 minutes,United Kingdom,English,"£180,000[1]",0.00000,0,1


We give a rank to the Running time of the film:

- R_time = 0.55  if  Running time in [60, 90] minutes
- R_time = 0.75  if  Running time in [60, 90] minutes
- R_time = 0.35 if  Running time > 120 minutes
- R_time = 0.05 if  Running time < 60 minutes

We multiply this factor and the tfidf:

- R_time * tfidf

And in the end we display only the films with the language
chosen by the user ranked by duration time.

In [320]:
# function for defining running time ranking
def R_time_score(row):
    try:
        time = int(row['Running time'].split()[0])
        if time < 60:
            return 0.1
        elif time > 120:
            return 0.45
        elif 60 <= time < 90:
            return 0.60
        elif 90 <= time <= 120:
            return 0.90
    except:
        return 0

In [321]:
search2['rank_duration'] = search2.apply(R_time_score, axis = 1)

In [306]:
search2

,Unnamed: 0,film_id,Wikipedia_link,title,intro,plot,film_name,Directed by,Produced by,Written by,...,Music by,Release date,Running time,Country,Language,Budget,Similarity,Engine1,sel_lan,rank_duration
0,0,0,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Love by the Light of the Moon,Edwin S. Porter,NaN,NaN,...,NaN,1901 (1901),NaN,United States,Silent,NaN,0.00000,0,0,0.00
1,1,1,https://en.wikipedia.org/wiki/The_Martyred_Pre...,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",The Martyred Presidents,Edwin S. Porter,NaN,NaN,...,NaN,1901,1 minutes,United States,English,NaN,0.00000,0,1,0.05
2,2,2,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...","Terrible Teddy, the Grizzly King","Terrible Teddy, the Grizzly King is a 1901 Ame...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0,0,0.00
3,3,3,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,Jack and the Beanstalk (1902 film),Jack and the Beanstalk is a 1902 American shor...,In this earliest known adaptation of the class...,Jack and the Beanstalk (1902 film),George S. FlemingEdwin S. Porter,Edison Studios,NaN,...,NaN,1902,10 minutes,United States,English,NaN,0.00000,0,1,0.05
4,4,4,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent f...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland (1903 film),Cecil HepworthPercy Stow,Cecil M. HepworthHerman Casler (exec. producer...,Lewis Carroll (book)Cecil M. Hepworth,...,NaN,17 October 1903 (1903-10-17),approx. 8:19 minutes,United Kingdom,Silent,NaN,0.57735,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995,29995,https://en.wikipedia.org/wiki/Paarai_(film),Paarai (film),Paarai (The Rock) is a 2003 Indian Tamil-langu...,Durairaj (Sarath Kumar) is a lorry driver who ...,Paarai (film),K.S.Ravikumar,S.S.Durairaj,NaN,...,Sabesh Murali,13 June 2003 (2003-06-13),170 minutes,India,Tamil,NaN,0.00000,0,0,0.35
29996,29996,29996,https://en.wikipedia.org/wiki/Jayam_(2003_film),Jayam (2003 film),Jayam (English: Victory) is a 2003 Indian Tami...,The story is centered in a village called Push...,Jayam (2003 film),M. Raja,M. VaralakshmiEditor Mohan,Prasanna Kumar (dialogues),...,R. P. Patnaik,20 June 2003 (2003-06-20),NaN,India,Tamil,NaN,0.57735,0,0,0.00
29997,29997,29997,https://en.wikipedia.org/wiki/Kaadhal_Kondein,Kaadhal Kondein,Kaadhal Kondein (also spelled as Kadhal Konden...,"Vinod (Dhanush), who has grown up under the ca...",Kaadhal Kondein,Selvaraghavan,Dr. K. Vimalageetha,Selvaraghavan,...,Yuvan Shankar Raja,4 July 2003,185 minutes,India,Tamil,NaN,0.00000,0,0,0.35
29998,29998,29998,https://en.wikipedia.org/wiki/Kovilpatti_Veera...,Kovilpatti Veeralakshmi,Kovilpatti Veeralakshmi is a 2003 Indian Tamil...,The film is about a brave woman who is battlin...,Kovilpatti Veeralakshmi,K. Rajeshwar,NaN,K. Rajeshwar,...,Adithyan,4 July 2003 (2003-07-04),192 minutes,India,Tamil,NaN,0.00000,0,0,0.35


In [322]:
def cosSim_3(row):
    film_vector = []
    query_vector = []
    for elem in cleaned_query3:
        if(elem in tfidf_vocabulary):
            if(row['film_id'] in tfidf_vocabulary[elem]):
                film_vector.append(tfidf_vocabulary[elem][row['film_id']]*row['rank_duration'])
                query_vector.append(1)
            else:
                film_vector.append(0)
                query_vector.append(1)
        else:
            film_vector.append(0)
            query_vector.append(1)
    
    cos_sim = cosine_similarity([film_vector], [query_vector])[0][0]
    return cos_sim

In [323]:
query3 = 'Disney movies 2019'
cleaned_query3 = clean_film(query3)
query_vector3 = tfidf_query(cleaned_query)
search2['Similarity'] = search2.apply(cosSim_3, axis = 1)
se3 = h[['title','Wikipedia_link', 'intro', 'Similarity']]

In [324]:
se3[se3['Similarity'] > 0.8].sort_values(by = ['Similarity'], ascending = False)

,title,Wikipedia_link,intro,Similarity
12266,Thumbelina (1994 film),https://en.wikipedia.org/wiki/Thumbelina_(1994...,Thumbelina (also known as Hans Christian Ander...,0.927540
25560,Jaanam Samjha Karo,https://en.wikipedia.org/wiki/Jaanam_Samjha_Karo,"Jaanam Samjha Karo (Hindi: जानम समझा करो, Urdu...",0.816363
16094,The Apparition,https://en.wikipedia.org/wiki/The_Apparition,The Apparition is a 2012 American-German super...,0.816363
16430,Frozen (2013 film),https://en.wikipedia.org/wiki/Frozen_(2013_film),Frozen is a 2013 American 3D computer-animated...,0.815474
14276,Scary Movie 3,https://en.wikipedia.org/wiki/Scary_Movie_3,Scary Movie 3 is a 2003 American horror sci-fi...,0.812547
10614,Benji the Hunted,https://en.wikipedia.org/wiki/Benji_the_Hunted,Benji the Hunted is a 1987 American children's...,0.812547
1976,She (1935 film),https://en.wikipedia.org/wiki/She_(1935_film),She is a 1935 American film produced by Merian...,0.812173
12311,The Big Green,https://en.wikipedia.org/wiki/The_Big_Green,The Big Green is an American 1995 family sport...,0.812173
9876,Annie (1982 film),https://en.wikipedia.org/wiki/Annie_(1982_film),Annie is a 1982 American musical comedy-drama ...,0.812173
12119,Ed Wood (film),https://en.wikipedia.org/wiki/Ed_Wood_(film),Ed Wood is a 1994 American biographical comedy...,0.812173


### Tuples for search query 3

In [326]:
list_tuples_3 = []
for i in range(30000):
    list_tuples_3.append((list(search2[search2['film_id'] == i]['Similarity'])[0], i))

In [327]:
heapq.heapify(list_tuples_3)
largest_sim_3 = nlargest(15, list_tuples_3)

In [328]:
res_sim_3 = pd.DataFrame(columns = ['title', 'intro', 'Wikipedia_link', 'Similarity'])
for elem in largest_sim_3:
    res_sim_3 = res_sim_3.append(h[h['film_id'] == elem[1]][['title', 'intro', 'Wikipedia_link', 'Similarity']])
res_sim_3

,title,intro,Wikipedia_link,Similarity
12266,Thumbelina (1994 film),Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,0.927540
25560,Jaanam Samjha Karo,"Jaanam Samjha Karo (Hindi: जानम समझा करो, Urdu...",https://en.wikipedia.org/wiki/Jaanam_Samjha_Karo,0.816363
16094,The Apparition,The Apparition is a 2012 American-German super...,https://en.wikipedia.org/wiki/The_Apparition,0.816363
16430,Frozen (2013 film),Frozen is a 2013 American 3D computer-animated...,https://en.wikipedia.org/wiki/Frozen_(2013_film),0.815474
14276,Scary Movie 3,Scary Movie 3 is a 2003 American horror sci-fi...,https://en.wikipedia.org/wiki/Scary_Movie_3,0.812547
10614,Benji the Hunted,Benji the Hunted is a 1987 American children's...,https://en.wikipedia.org/wiki/Benji_the_Hunted,0.812547
1976,She (1935 film),She is a 1935 American film produced by Merian...,https://en.wikipedia.org/wiki/She_(1935_film),0.812173
25221,Jo Jeeta Wohi Sikandar,"Jo Jeeta Wohi Sikandar (transl. Whoever wins, ...",https://en.wikipedia.org/wiki/Jo_Jeeta_Wohi_Si...,0.812173
22259,Mulan (2009 film),Mulan: Rise of a Warrior (simplified Chinese: ...,https://en.wikipedia.org/wiki/Mulan_(2009_film),0.812173
21587,Clearcut (film),Clearcut is a 1991 Canadian drama film about c...,https://en.wikipedia.org/wiki/Clearcut_(film),0.812173
